<a href="https://colab.research.google.com/github/HeyJae-zero/Final-Team9/blob/main/%EC%8B%9C%EB%A6%AC%EC%A6%88%EB%B3%84_%ED%9D%A5%ED%96%89%EC%B2%99%EB%8F%84_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 사용한 통계 머신

1. 부호 검정: 누가 더 자주 이겼나를 보기 위해 사용 (승패 여부)
2. 윌콕슨: 얼마나 크게 이겼나를 보기 위해 사용 (크기 확인)

### 왜 비모수 검정을 사용했는가?
1. 이산형 데이터: y_result가 0~3의 순서형 척도
2. 제한된 값의 범위: 차이(diff)가 -3~3 사이의 정수만 가능
3. 순서형 특성: 등간격 가정이 애매함 (1점 차이 = 2점 차이?)
4. 분포의 치우침 가능성: 대부분 차이가 작을 가능성

### 결과표 보는 법

- sign test p값(one-sided): 후속이 더 자주 이기냐?
- wilcoxon p값(one-sided): 후속이 크게 이기는 경향도 있냐?
- diff 중앙값/평균: 평균적으로 얼마나 차이 나는지
- wins_sequel / wins_prior / ties_or_na: 후속 승/이전 승/비김의 숫자

In [1]:
# -*- coding: utf-8 -*-
"""
TMDB_processed_final_with_series.csv 기반
'계층적 비교(1→2, 2→3, …; 중간 없으면 1→3처럼 다음 유효 편과 비교)'에 대해
부호검정(Sign test)과 윌콕슨 부호순위검정(Wilcoxon signed-rank)을 함께 수행하여
방향(누가 더 우세?)과 차이의 크기(효과 크기, 중앙값/평균)를 모두 보여주는 스크립트.

- y_result 컬럼은 CSV에 이미 존재한다고 가정 (0~3의 순서형 점수)
- Avengers(1998)는 avengers 시리즈에서 제외
- 결과 저장(현재 작업 폴더):
    1) pairs_level_details.csv      : 모든 시리즈의 (i→j) 페어 상세 및 diff
    2) level_tests_summary.csv      : 각 전이 레벨(예: 1→2, 2→3, 1→3 등)별 검정 요약
    3) overall_tests_summary.csv    : 전체 페어에 대한 검정 요약
"""

import os
import math
import numpy as np
import pandas as pd
from datetime import datetime

# SciPy가 있으면 정확 검정 사용, 없으면 대체 절차(정규근사/부호검정)로 동작
try:
    from scipy.stats import binomtest, wilcoxon
    SCIPY = True
except Exception:
    SCIPY = False

# ---------- (0) 파일 로드 ----------
PATH = "/content/TMDB_processed_final_with_series.csv"
assert os.path.exists(PATH), f"파일이 없습니다: {PATH}"
df = pd.read_csv(PATH)

# ---------- (1) 컬럼명 설정 (필요시 아래만 수정) ----------
title_col = "title"
series_col = "series_group"
release_date_col = "release_date"
yresult_col = "y_result"

# ---------- (2) 연도 추출(정렬용) ----------
df["year_extracted"] = pd.to_datetime(df[release_date_col], errors="coerce").dt.year

# ---------- (3) Avengers(1998) 시리즈 제외 ----------
mask_avengers_1998 = (
    df[title_col].astype(str).str.strip().str.lower().eq("the avengers")
    & (df["year_extracted"] == 1998)
    & df[series_col].astype(str).str.lower().str.contains("avengers", na=False)
)
df.loc[mask_avengers_1998, series_col] = np.nan
print(f"[INFO] 'The Avengers (1998)' 시리즈 그룹 제외 행 수: {int(mask_avengers_1998.sum())}")

# ---------- (4) 시리즈 내 정렬 및 installment 부여 ----------
series_df = df[df[series_col].notna()].copy()
series_df = series_df.sort_values(
    by=[series_col, "year_extracted", release_date_col, title_col],
    ascending=[True, True, True, True],
)
series_df["installment"] = series_df.groupby(series_col).cumcount() + 1
series_df[yresult_col] = pd.to_numeric(series_df[yresult_col], errors="coerce")

# ---------- (5) '가장 가까운 다음 편'과 짝짓기
# 정렬된 순서에서 인접 행끼리 묶기 → 중간 편이 비어 있으면 자연스럽게 1→3 같은 페어가 만들어짐
pairs = []
for grp, g in series_df.groupby(series_col, sort=False):
    g = g.sort_values("installment")
    rows = list(g.itertuples(index=False))
    for a, b in zip(rows[:-1], rows[1:]):   # 인접 행끼리 페어링
        i_inst = int(getattr(a, "installment"))
        j_inst = int(getattr(b, "installment"))
        i_y = getattr(a, yresult_col)
        j_y = getattr(b, yresult_col)
        pairs.append({
            "series_group": grp,
            "i_installment": i_inst,
            "j_installment": j_inst,
            "level_label": f"{i_inst}->{j_inst}",  # 계층적 전이 라벨
            "i_title": getattr(a, title_col),
            "j_title": getattr(b, title_col),
            "i_year": int(getattr(a, "year_extracted")) if pd.notna(getattr(a, "year_extracted")) else None,
            "j_year": int(getattr(b, "year_extracted")) if pd.notna(getattr(b, "year_extracted")) else None,
            "i_y_result": i_y,
            "j_y_result": j_y,
        })

pairs_df = pd.DataFrame(pairs).dropna(subset=["i_y_result", "j_y_result"])
pairs_df["i_y_result"] = pairs_df["i_y_result"].astype(int)
pairs_df["j_y_result"] = pairs_df["j_y_result"].astype(int)
pairs_df["diff"] = pairs_df["j_y_result"] - pairs_df["i_y_result"]
pairs_df["win"] = np.where(pairs_df["diff"] > 0, 1, np.where(pairs_df["diff"] < 0, -1, 0))

print(f"[INFO] 생성된 계층 페어 수: {len(pairs_df)}")
if len(pairs_df) == 0:
    raise SystemExit("[ERROR] 유효한 페어가 없습니다. 시리즈/컬럼 구성을 확인해 주세요.")

# ---------- (6) 검정 함수 ----------
def sign_test(win_series):
    """부호검정(Sign test)을 이항검정으로 수행: H1: 후속 승 비율 > 0.5"""
    s = win_series.dropna()
    s = s[s != 0]  # 동률/NA 제외
    n_plus = int((s == 1).sum())
    n_minus = int((s == -1).sum())
    n = n_plus + n_minus
    if n == 0:
        return {"n_eff": 0, "wins_sequel": n_plus, "wins_prior": n_minus, "p_one": np.nan, "p_two": np.nan}
    if SCIPY:
        p_two = binomtest(n_plus, n, 0.5, alternative="two-sided").pvalue
        p_one = binomtest(n_plus, n, 0.5, alternative="greater").pvalue
    else:
        # 정규근사(간이)
        phat = n_plus / n
        se = math.sqrt(0.25 / n)
        z = (phat - 0.5) / se
        p_one = 0.5 * (1 - math.erf(z / math.sqrt(2)))
        p_two = min(1.0, 2 * p_one)
    return {"n_eff": n, "wins_sequel": n_plus, "wins_prior": n_minus, "p_one": p_one, "p_two": p_two}

def wilcoxon_or_sign(diff_series):
    """Wilcoxon 한쪽/양쪽 p; SciPy 없으면 부호검정 대체"""
    d = diff_series.dropna()
    d = d[d != 0]
    if len(d) == 0:
        return {"n_eff": 0, "stat": np.nan, "p_one": np.nan, "p_two": np.nan}
    if SCIPY:
        stat_g, p_one = wilcoxon(d, alternative="greater", zero_method="wilcox")
        stat_t, p_two = wilcoxon(d, alternative="two-sided", zero_method="wilcox")
        return {"n_eff": int(len(d)), "stat": float(stat_g), "p_one": float(p_one), "p_two": float(p_two)}
    else:
        # 부호검정으로 대체
        n_plus = int((d > 0).sum()); n_minus = int((d < 0).sum()); n = n_plus + n_minus
        if n == 0:
            return {"n_eff": 0, "stat": np.nan, "p_one": np.nan, "p_two": np.nan}
        phat = n_plus / n
        se = math.sqrt(0.25 / n)
        z = (phat - 0.5) / se
        p_one = 0.5 * (1 - math.erf(z / math.sqrt(2)))
        p_two = min(1.0, 2 * p_one)
        return {"n_eff": n, "stat": np.nan, "p_one": p_one, "p_two": p_two}

def summarize_block(block_df, label="OVERALL"):
    """한 전이 레벨(예: 1→2) 또는 전체에 대해 Sign/Wilcoxon + 효과크기 요약"""
    wins = block_df["win"].fillna(0)
    sign_res = sign_test(wins)

    wres = wilcoxon_or_sign(block_df["diff"])

    # 효과 크기(차이의 크기) 요약: 중앙값/평균/표준편차, 양/음/0 개수
    diff = block_df["diff"]
    diff_nz = diff[diff != 0].dropna()
    summary = {
        "level_label": label,
        "pairs_total": int(len(block_df)),
        "pairs_used_sign": int(sign_res["n_eff"]),
        "pairs_used_wilcoxon": int(wres["n_eff"]),
        "wins_sequel": int(sign_res["wins_sequel"]),
        "wins_prior": int(sign_res["wins_prior"]),
        "ties_or_na": int((wins == 0).sum()),
        "diff_median": float(np.nanmedian(diff)) if len(diff) > 0 else np.nan,
        "diff_mean": float(np.nanmean(diff)) if len(diff) > 0 else np.nan,
        "diff_std": float(np.nanstd(diff, ddof=1)) if len(diff) > 1 else np.nan,
        "diff_pos_count": int((diff > 0).sum()),
        "diff_neg_count": int((diff < 0).sum()),
        "diff_zero_count": int((diff == 0).sum()),
        "sign_p_one_sided(H1: sequel > prior)": sign_res["p_one"],
        "sign_p_two_sided": sign_res["p_two"],
        "wilcoxon_stat": wres["stat"],
        "wilcoxon_p_one_sided(H1: sequel > prior)": wres["p_one"],
        "wilcoxon_p_two_sided": wres["p_two"],
    }
    return summary

# ---------- (7) 레벨별 요약 ----------
level_summaries = []
for lvl, g in pairs_df.groupby("level_label", sort=False):
    level_summaries.append(summarize_block(g, label=lvl))

level_summary_df = pd.DataFrame(level_summaries).sort_values(
    by=["level_label", "pairs_total"], ascending=[True, False]
)

# ---------- (8) 전체(OVERALL) 요약 ----------
overall_summary_df = pd.DataFrame([summarize_block(pairs_df, label="OVERALL")])

# ---------- (9) 저장 ----------
pairs_df.to_csv("pairs_level_details.csv", index=False)
level_summary_df.to_csv("level_tests_summary.csv", index=False)
overall_summary_df.to_csv("overall_tests_summary.csv", index=False)

print("[저장 완료] pairs_level_details.csv")
print("[저장 완료] level_tests_summary.csv")
print("[저장 완료] overall_tests_summary.csv")

# ---------- (10) 콘솔 표시(요약) ----------
print("\n=== OVERALL SUMMARY ===")
print(overall_summary_df.to_string(index=False))

print("\n=== LEVEL SUMMARY (head) ===")
print(level_summary_df.head(20).to_string(index=False))

[INFO] 'The Avengers (1998)' 시리즈 그룹 제외 행 수: 1
[INFO] 생성된 계층 페어 수: 362
[저장 완료] pairs_level_details.csv
[저장 완료] level_tests_summary.csv
[저장 완료] overall_tests_summary.csv

=== OVERALL SUMMARY ===
level_label  pairs_total  pairs_used_sign  pairs_used_wilcoxon  wins_sequel  wins_prior  ties_or_na  diff_median  diff_mean  diff_std  diff_pos_count  diff_neg_count  diff_zero_count  sign_p_one_sided(H1: sequel > prior)  sign_p_two_sided  wilcoxon_stat  wilcoxon_p_one_sided(H1: sequel > prior)  wilcoxon_p_two_sided
    OVERALL          362              155                  155           38         117         207          0.0  -0.298343  0.845026              38             117              207                                   1.0      1.478869e-10         2746.0                                       1.0          3.605934e-10

=== LEVEL SUMMARY (head) ===
level_label  pairs_total  pairs_used_sign  pairs_used_wilcoxon  wins_sequel  wins_prior  ties_or_na  diff_median  diff_mean  diff_std  diff_p